In [1]:
import os
from dotenv import load_dotenv
import requests
import json

In [2]:
# Load API Key
load_dotenv()
AIRDNA_TOKEN = os.getenv("AIRDNA_TOKEN")

In [3]:
# Same for All API Calls
HEADERS = {"access_token": AIRDNA_TOKEN}
HOST = "https://api.airdna.co/client/v1/"

In [5]:
# MARKET SEARCH
# {{HOST}}/client/{{VERSION_CLIENT}}/market/search?
# access_token={{CLIENT_TOKEN}}&term='https://www.airbnb.com/rooms/15807599'
# Country
# City
# Neighbourhood
# State (US Only)
# Metropolitan Statistical Area (US Only)
# Zip Code (US Only)

In [23]:
# Test Connection HERE <- Search Calls are Free
# API CALL - Market Search - Find Gatlinburg Market ID
endpoint = "market/search"
url = HOST + endpoint

params = {
    "access_token": AIRDNA_TOKEN,
    "term": "gatlinburg"
}


response = requests.request("GET", url,  
                            params=params)

In [24]:
# Get Market ID
gatlinburg_id = json['items'][0]['city']['id']

# Other information Available from This Response:
# - name, country, region, state, type, zipcode

In [25]:
gatlinburg_id

79116

In [4]:
# MARKET SUMMARY
# {{HOST}}/client/{{VERSION}}/market/summary?
# access_token={{CLIENT_TOKEN}}&city_id=59380&region_id=12341&currency=usd

# PARAMS
# city_id
# region_id
# currency

In [28]:
# API CALL - Market Summary - Get Info on Gatlinburg
endpoint = "market/summary"
url = HOST + endpoint

params = {
    "access_token": AIRDNA_TOKEN,
    "city_id": gatlinburg_id
}


response = requests.request("GET", url,  
                            params=params)

In [45]:
# Jasonify Results
json = response.json()

## Parcing Through Available Data at Market Summary Enpoint

In [124]:
# Data Frame Dictionary
dfd = {}

In [125]:
# Market - Summary - Request_Info
# COLS: city_id
city_id = json['request_info']['city_id']
dfd.update({'city_id': city_id})
city_id

79116

In [126]:
# Market - Summary - Data - Calendar Months
# adr -> Average Daily Rate
# COLS: occ, adr, revenue
ent_stats = json['data']['calendar_months']['room_type']['entire_place']
ent_stats
ent_stats_dict = {
    'occ': ent_stats['occ']['50th_percentile'],
    'adr': ent_stats['adr']['50th_percentile'],
    'revenue': ent_stats['revenue']['50th_percentile']
                 }
dfd.update(ent_stats_dict)
ent_stats

{'occ': {'50th_percentile': 0.741935483870968},
 'adr': {'50th_percentile': 263.0},
 'revenue': {'50th_percentile': 4745.0}}

In [127]:
# Market - Summary - Data - Host_Info - Hosts
# COLS: total_hosts, superhosts, multi_unit_hosts, single_unit_hosts
host_counts = json['data']['host_info']['hosts']
dfd.update(host_counts)
host_counts

{'total_hosts': 642,
 'superhosts': 343,
 'multi_unit_hosts': 197,
 'single_unit_hosts': 445}

In [128]:
# Market - Summary - Data - Host_Info - Host_Properties
# COLS: multi_host_properties, total_properties, single_host_properties
property_counts = json['data']['host_info']['host_properties']
dfd.update(property_counts)
property_counts

{'multi_host_properties': 3104,
 'total_properties': 3549,
 'single_host_properties': 445}

In [129]:
# Market - Summary - Data - Rental_Activity - Available
# COLS:
# TODO: ?- What is '10-12', '1-3', and such? -?
# They are either length-of-stay or they are n-people
available = json['data']['rental_activity']['available']
available_dict = {
    '10_12_available': available['10-12'],
    '1_3_available': available['1-3'],
    '7_9_available': available['7-9'],
    '4_6_available': available['4-6']
                 }

dfd.update(available_dict)
available

{'10-12': 2054, '1-3': 1066, '7-9': 667, '4-6': 816}

In [130]:
# Market - Summary - Data - Rental_Activity - Booked
# COLS:
# TODO: ?- What is '10-12', '1-3', and such? -?
# They are either length-of-stay or they are n-people
booked = json['data']['rental_activity']['booked']
booked_dict = {
    '10_12_booked': booked['10-12'],
    '1_3_booked': booked['1-3'],
    '7_9_booked': booked['7-9'],
    '4_6_booked': booked['4-6']
                 }
dfd.update(booked_dict)
booked

{'10-12': 435, '1-3': 1764, '7-9': 1230, '4-6': 1174}

In [131]:
# Market - Summary - Data - Rental Counts - Counts - Private_Room
# COLS: n_private_rooms
n_private_rooms = json['data']['rental_counts']['counts']['private_room']['all']
dfd.update({'n_private_rooms': n_private_rooms})
json['data']['rental_counts']['counts']['private_room']

{'0': 14.0, '1': 73.0, '2': 15.0, '3': 2.0, 'all': 104.0}

In [132]:
# Market - Summary - Data - Rental Counts - Counts - Entire Place by n_rooms
# COLS: rms0_rntl_cnt, rms1_rntl_cnt, rms3_rntl_cnt
#       rms4_rntl_cnt, rms5plus_rntl_cnt, n_entire_place
count_by_n_rooms = json['data']['rental_counts']['counts']['entire_place']
count_by_n_rooms_dict = {
    'n_rooms_0': count_by_n_rooms['0'],
    'n_rooms_1': count_by_n_rooms['1'],
    'n_rooms_2': count_by_n_rooms['2'],
    'n_rooms_3': count_by_n_rooms['3'],
    'n_rooms_4': count_by_n_rooms['4'],
    'n_rooms_5plus': count_by_n_rooms['5'],
    'tot_count_entire_place': count_by_n_rooms['all'],
                        }

dfd.update(count_by_n_rooms_dict)
count_by_n_rooms

{'0': 112.0,
 '1': 815.0,
 '2': 1213.0,
 '3': 643.0,
 '4': 317.0,
 '5': 344.0,
 'all': 3444.0}

In [133]:
# Market - Summary - Data - Rental Counts - Average
# COLS: average_n_bedrooms, average_accomodates
average = json['data']['rental_counts']['average']
average_dict = {
    'avg_n_rooms': average['bedrooms'],
    'avg_accommodates': average['accommodates']
}
dfd.update(average_dict)

In [134]:
dfd

{'city_id': 79116,
 'occ': 0.741935483870968,
 'adr': 263.0,
 'revenue': 4745.0,
 'total_hosts': 642,
 'superhosts': 343,
 'multi_unit_hosts': 197,
 'single_unit_hosts': 445,
 'multi_host_properties': 3104,
 'total_properties': 3549,
 'single_host_properties': 445,
 '10_12_available': 2054,
 '1_3_available': 1066,
 '7_9_available': 667,
 '4_6_available': 816,
 '10_12_booked': 435,
 '1_3_booked': 1764,
 '7_9_booked': 1230,
 '4_6_booked': 1174,
 'n_private_rooms': 104.0,
 'n_rooms_0': 112.0,
 'n_rooms_1': 815.0,
 'n_rooms_2': 1213.0,
 'n_rooms_3': 643.0,
 'n_rooms_4': 317.0,
 'n_rooms_5plus': 344.0,
 'tot_count_entire_place': 3444.0,
 'avg_n_rooms': 2.5,
 'avg_accommodates': 7.6}

In [135]:
import pandas as pd

In [139]:
gat_df_row = pd.Series(dfd)
gat_df_row

city_id                   79116.000000
occ                           0.741935
adr                         263.000000
revenue                    4745.000000
total_hosts                 642.000000
superhosts                  343.000000
multi_unit_hosts            197.000000
single_unit_hosts           445.000000
multi_host_properties      3104.000000
total_properties           3549.000000
single_host_properties      445.000000
10_12_available            2054.000000
1_3_available              1066.000000
7_9_available               667.000000
4_6_available               816.000000
10_12_booked                435.000000
1_3_booked                 1764.000000
7_9_booked                 1230.000000
4_6_booked                 1174.000000
n_private_rooms             104.000000
n_rooms_0                   112.000000
n_rooms_1                   815.000000
n_rooms_2                  1213.000000
n_rooms_3                   643.000000
n_rooms_4                   317.000000
n_rooms_5plus            